# Markov chain-based approach for the music generation

In [7]:
import numpy as np
import pandas as pd
from collections import Counter
np.random.seed(42)

# read file
data = pd.read_csv('data/Liverpool_band_chord_sequence.csv')

## Generate bigrams

In [8]:
n = 2
chords = data['chords'].values
ngrams = zip(*[chords[i:] for i in range(n)])
bigrams = [" ".join(ngram) for ngram in ngrams]

bigrams[:5]

['F Em7', 'Em7 A7', 'A7 Dm', 'Dm Dm7', 'Dm7 Bb']

## Predict next state with Markov chain

In [9]:
def predict_next_state(chord:str, bigrams:list=bigrams):
    """Predict next chord based on current state."""
    # create list of bigrams which stats with current chord
    bigrams_with_current_chord = [bigram for bigram in bigrams if bigram.split(' ')[0]==chord]
    # count appearance of each bigram
    count_appearance = dict(Counter(bigrams_with_current_chord))
    # convert apperance into probabilities
    for ngram in count_appearance.keys():
        count_appearance[ngram] = count_appearance[ngram]/len(bigrams_with_current_chord)
    # create list of possible options for the next chord
    # uses second chord from bigram, because first one is current chord (see line 4)
    options = [key.split(' ')[1] for key in count_appearance.keys()]
    # create  list of probability distribution
    probabilities = list(count_appearance.values())
    # return random prediction
    return np.random.choice(options, p=probabilities)

In [10]:
# example
predict_next_state('F')

'C'

## Generate sequence

In [23]:
def generate_sequence(chord:str=None, bigrams:list=bigrams, length:int=20):
    """Generate sequence of defined length."""
    # create list to store future chords
    chords = []
    for n in range(length):
        # append next chord for the list
        chords.append(predict_next_state(chord, bigrams))
        # use last chord in sequence to predict next chord
        chord = chords[-1]
    return chords

In [44]:
start_chord = 'Dm'
generated_seq = generate_sequence(start_chord)
print(generated_seq)

['Dm7', 'G7', 'Bb', 'F', 'C', 'Bb', 'F', 'Em7', 'A7', 'Dm', 'Gm6', 'C7', 'F', 'Fsus4', 'F', 'Em7', 'A7', 'Dm', 'C', 'Dm7']


In [45]:
from musicpy import *

# create piece with first chord
piece = piece([C(start_chord)], instruments=[1], start_times=[0], channels=[0]) # general midi instruments

# append generated chords to the piece
for i, chord in enumerate(generated_seq):
    t = track(C(chord), instrument=1, start_time=(i+1)*0.3, channel=0) # general midi instruments
    piece.append(t)

# play piece
piece.play()
